## Code to generate tradingview images from chartimg api

In [4]:
import requests  # For making API calls
from datetime import datetime, date  # For date calculations
from IPython.display import Image, display  # For displaying images in the notebook
import os  # For file operations
import time  # For adding delays between API calls
import json  # For saving/loading usage data

# Define multiple API keys and endpoint URL
API_KEYS = [
    "VMpljkjwQu8Sr1vQOvSmIwmeNpnlmXP6euqzWvje", #onlystockmarket8
    "RHHLaR7k9h3p3mww6idK18S9ycvX7Pwt4p48IlAn", #spsandy715
    # "your_second_api_key_here",  # Add more API keys as needed
    # "your_third_api_key_here",
]

ENDPOINT_URL = "https://api.chart-img.com/v2/tradingview/advanced-chart"
DAILY_LIMIT = 50  # API calls per day per key
USAGE_FILE = "api_usage.json"  # File to track API usage

# Ensure the 'images' folder exists
os.makedirs("images", exist_ok=True)

class APIKeyManager:
    def __init__(self, api_keys, daily_limit):
        self.api_keys = api_keys
        self.daily_limit = daily_limit
        self.current_key_index = 0
        self.usage_data = self.load_usage_data()
        
    def load_usage_data(self):
        """Load API usage data from file"""
        try:
            if os.path.exists(USAGE_FILE):
                with open(USAGE_FILE, 'r') as f:
                    data = json.load(f)
                # Clean old dates (keep only today's data)
                today = str(date.today())
                for key in data:
                    if today not in data[key]:
                        data[key][today] = 0
                    # Remove old dates
                    data[key] = {today: data[key].get(today, 0)}
                return data
            else:
                return {}
        except Exception as e:
            print(f"Error loading usage data: {e}")
            return {}
    
    def save_usage_data(self):
        """Save API usage data to file"""
        try:
            with open(USAGE_FILE, 'w') as f:
                json.dump(self.usage_data, f, indent=2)
        except Exception as e:
            print(f"Error saving usage data: {e}")
    
    def get_current_key(self):
        """Get the current API key"""
        if self.current_key_index >= len(self.api_keys):
            return None
        return self.api_keys[self.current_key_index]
    
    def get_current_usage(self, api_key=None):
        """Get current usage for a specific API key"""
        if api_key is None:
            api_key = self.get_current_key()
        
        if api_key not in self.usage_data:
            self.usage_data[api_key] = {}
        
        today = str(date.today())
        if today not in self.usage_data[api_key]:
            self.usage_data[api_key][today] = 0
            
        return self.usage_data[api_key][today]
    
    def increment_usage(self, api_key=None):
        """Increment usage count for current API key"""
        if api_key is None:
            api_key = self.get_current_key()
            
        today = str(date.today())
        if api_key not in self.usage_data:
            self.usage_data[api_key] = {}
        if today not in self.usage_data[api_key]:
            self.usage_data[api_key][today] = 0
            
        self.usage_data[api_key][today] += 1
        self.save_usage_data()
    
    def can_make_request(self, api_key=None):
        """Check if we can make a request with current API key"""
        if api_key is None:
            api_key = self.get_current_key()
        
        if api_key is None:
            return False
            
        current_usage = self.get_current_usage(api_key)
        return current_usage < self.daily_limit
    
    def switch_to_next_key(self):
        """Switch to next available API key"""
        self.current_key_index += 1
        if self.current_key_index >= len(self.api_keys):
            return False  # No more keys available
        
        current_key = self.get_current_key()
        current_usage = self.get_current_usage(current_key)
        print(f"Switched to API key #{self.current_key_index + 1} (Usage: {current_usage}/{self.daily_limit})")
        return True
    
    def get_best_available_key(self):
        """Find the API key with lowest usage"""
        best_key = None
        best_index = -1
        lowest_usage = float('inf')
        
        for i, key in enumerate(self.api_keys):
            usage = self.get_current_usage(key)
            if usage < self.daily_limit and usage < lowest_usage:
                lowest_usage = usage
                best_key = key
                best_index = i
        
        if best_key:
            self.current_key_index = best_index
            return best_key
        return None
    
    def print_usage_summary(self):
        """Print usage summary for all API keys"""
        print("\n=== API Usage Summary ===")
        today = str(date.today())
        for i, key in enumerate(self.api_keys):
            usage = self.get_current_usage(key)
            status = "AVAILABLE" if usage < self.daily_limit else "LIMIT REACHED"
            masked_key = key[:8] + "..." + key[-4:] if len(key) > 12 else key
            print(f"Key #{i+1} ({masked_key}): {usage}/{self.daily_limit} - {status}")
        print("========================\n")

# Initialize API key manager
api_manager = APIKeyManager(API_KEYS, DAILY_LIMIT)

# Function to generate the payload
def create_payload(symbol, interval, period, studies):
    return {
        "symbol": symbol,
        "interval": interval,
        "range": period,
        "studies": studies
    }

# Function to save the chart image with new naming convention
def save_chart_image(response, symbol, interval, symbol_index, config_index):
    # Extract symbol name (remove exchange prefix)
    symbol_name = symbol.split(':')[-1] if ':' in symbol else symbol
    today_str = datetime.now().strftime("%Y-%m-%d")
    image_filename = f"images/{symbol_index}.{config_index} - {symbol_name}_{interval}_{today_str}.png"
    
    with open(image_filename, "wb") as file:
        file.write(response.content)
    return image_filename

# Function to display the chart image
def display_chart_image(image_filename):
    if os.path.exists(image_filename):
        display(Image(filename=image_filename))
    else:
        print(f"Chart image file '{image_filename}' not found.")

# Function to make API request with automatic key rotation
def make_api_request(payload, max_retries=3):
    """Make API request with automatic key rotation on failure"""
    
    for attempt in range(max_retries):
        # Get current available API key (stick with current key until exhausted)
        current_key = api_manager.get_best_available_key()
        
        if not current_key:
            print("❌ All API keys have reached their daily limits!")
            return None
        
        # Prepare headers
        headers = {
            "x-api-key": current_key,
            "content-type": "application/json"
        }
        
        current_usage = api_manager.get_current_usage(current_key)
        print(f"Using API key #{api_manager.current_key_index + 1} (Usage: {current_usage}/{DAILY_LIMIT})")
        
        try:
            response = requests.post(ENDPOINT_URL, headers=headers, json=payload, timeout=30)
            
            # Check for rate limit errors
            if response.status_code == 429 or "rate limit" in response.text.lower() or "quota" in response.text.lower():
                print(f"⚠️ Rate limit reached for API key #{api_manager.current_key_index + 1}")
                # Mark current key as exhausted
                api_manager.usage_data[current_key][str(date.today())] = DAILY_LIMIT
                api_manager.save_usage_data()
                
                if api_manager.switch_to_next_key():
                    print(f"🔄 Switching to next API key...")
                    continue
                else:
                    print("❌ No more API keys available!")
                    return None
            
            elif response.status_code == 200:
                # Success! Increment usage counter
                api_manager.increment_usage(current_key)
                print(f"✅ Request successful!")
                return response
            
            else:
                print(f"❌ API request failed. Status Code: {response.status_code}")
                print(f"Response: {response.text}")
                if attempt < max_retries - 1:
                    print(f"Retrying... (Attempt {attempt + 2}/{max_retries})")
                    time.sleep(2)
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Request error: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying... (Attempt {attempt + 2}/{max_retries})")
                time.sleep(2)
    
    return None

# Function to generate chart for a single configuration
def generate_single_chart(symbol, interval, period, studies, symbol_index, config_index):
    payload = create_payload(symbol, interval, period, studies)
    print(f"\n📊 Generating {interval} chart for {symbol}...")
    
    response = make_api_request(payload)
    
    if response and response.status_code == 200:
        image_filename = save_chart_image(response, symbol, interval, symbol_index, config_index)
        print(f"💾 Chart image saved successfully as {image_filename}")
        # display_chart_image(image_filename)
        return True
    else:
        print(f"❌ Failed to generate {interval} chart for {symbol}")
        return False

# Main function to generate chart images for multiple symbols
def generate_images_batch(stocks):
    # Print initial usage summary
    api_manager.print_usage_summary()
    
    # Calculate total requests needed
    total_requests = len(stocks) * 3  # 3 configurations per symbol
    print(f"📋 Total requests needed: {total_requests}")
    
    # Check if we have enough quota
    total_available = sum(DAILY_LIMIT - api_manager.get_current_usage(key) for key in API_KEYS)
    if total_requests > total_available:
        print(f"⚠️ Warning: You need {total_requests} requests but only have {total_available} available across all keys!")
        proceed = input("Do you want to proceed anyway? (y/n): ")
        if proceed.lower() != 'y':
            print("Operation cancelled.")
            return
    
    # Define chart configurations
    configurations = [
        {
            "interval": "1D",
            "period": "6M",
            "studies": [
                {
                    "name": "Volume",
                    "forceOverlay": True
                },
                {
                    "name": "Moving Average",
                    "input": {
                        "length": 30
                    },
                    "override": {
                        "Plot.color": "rgba(173,216,230,1)"  # Light blue color
                    }
                }
            ],
            "config_index": 1
        },
        {
            "interval": "1W",
            "period": "5Y",
            "studies": [
                {
                    "name": "Volume",
                    "forceOverlay": True
                },
                {
                    "name": "Moving Average",
                    "input": {
                        "length": 50
                    },
                    "override": {
                        "Plot.color": "rgba(173,216,230,1)"  # Light blue color
                    }
                }
            ],
            "config_index": 2
        },
        {
            "interval": "1M",
            "period": "ALL",
            "studies": [
                {
                    "name": "Volume",
                    "forceOverlay": True
                }
            ],
            "config_index": 3
        }
    ]
    
    successful_requests = 0
    failed_requests = 0
    
    # Process each symbol
    for symbol_index, symbol in enumerate(stocks, 1):
        print(f"\n{'='*60}")
        print(f"🔄 Processing Symbol {symbol_index}/{len(stocks)}: {symbol}")
        print(f"{'='*60}")
        
        # Process each configuration for the current symbol
        for config in configurations:
            success = generate_single_chart(
                symbol=symbol,
                interval=config["interval"],
                period=config["period"],
                studies=config["studies"],
                symbol_index=symbol_index,
                config_index=config["config_index"]
            )
            
            if success:
                successful_requests += 1
            else:
                failed_requests += 1
            
            # Add a small delay between API calls
            time.sleep(1)
        
        print(f"✅ Completed processing {symbol}")
    
    # Final summary
    print(f"\n{'='*60}")
    print(f"📊 FINAL SUMMARY")
    print(f"{'='*60}")
    print(f"✅ Successful requests: {successful_requests}")
    print(f"❌ Failed requests: {failed_requests}")
    print(f"📈 Total processed: {successful_requests + failed_requests}")
    
    # Print final usage summary
    api_manager.print_usage_summary()

# Utility function to check current usage
def check_api_usage():
    """Check current API usage for all keys"""
    api_manager.print_usage_summary()

# Utility function to reset usage (use with caution!)
def reset_usage_data():
    """Reset usage data - USE WITH CAUTION!"""
    confirm = input("Are you sure you want to reset usage data? This cannot be undone! (type 'RESET' to confirm): ")
    if confirm == 'RESET':
        if os.path.exists(USAGE_FILE):
            os.remove(USAGE_FILE)
        print("Usage data has been reset.")
    else:
        print("Reset cancelled.")



In [ ]:
# Example usage
if __name__ == "__main__":
    # Check current usage before starting
    check_api_usage()
    
    # Define your list of stocks
    stocks = ['NSE:NIFTY', 'BSE:NITTAGELA','NSE:NAVA']
    
    # Generate images for all stocks
    generate_images_batch(stocks)
    
    print("\n🎉 Chart generation process completed!")


--- Processing Symbol 1: NSE:NIFTY ---
Generating 1D chart for NSE:NIFTY...
Chart image saved successfully as images/1.1 - NIFTY_1D_2025-09-09.png
Generating 1W chart for NSE:NIFTY...
Chart image saved successfully as images/1.2 - NIFTY_1W_2025-09-09.png
Generating 1M chart for NSE:NIFTY...
Chart image saved successfully as images/1.3 - NIFTY_1M_2025-09-09.png
Completed processing NSE:NIFTY
--------------------------------------------------

--- Processing Symbol 2: BSE:NITTAGELA ---
Generating 1D chart for BSE:NITTAGELA...
Chart image saved successfully as images/2.1 - NITTAGELA_1D_2025-09-09.png
Generating 1W chart for BSE:NITTAGELA...
Chart image saved successfully as images/2.2 - NITTAGELA_1W_2025-09-09.png
Generating 1M chart for BSE:NITTAGELA...
Chart image saved successfully as images/2.3 - NITTAGELA_1M_2025-09-09.png
Completed processing BSE:NITTAGELA
--------------------------------------------------

--- Processing Symbol 3: NSE:NN ---
Generating 1D chart for NSE:NN...
Faile

## Code to upload images to google drive and then to presentation automatically

In [13]:


from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account

# Configuration
drive_SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/presentations']
drive_SERVICE_ACCOUNT_FILE = '/Volumes/Work/Coding/Automation/gsheet_auto_onlystockmarket8.json'
drive_PARENT_FOLDER_ID = "1jd6ztmZL1JlBTRh8uzvDS-bBoSojY0XQ"
PRESENTATION_ID = "1h6rz2eed078IJi4ouUhmaJem-4QooDZxe_izoWwqQoQ"

# Standard slide dimensions in points (for 16:9 aspect ratio)
SLIDE_WIDTH_PT = 720  # 10 inches * 72 points/inch
SLIDE_HEIGHT_PT = 405  # 5.625 inches * 72 points/inch

def authenticate():
    """Authenticate and return credentials for Google APIs"""
    creds = service_account.Credentials.from_service_account_file(
        drive_SERVICE_ACCOUNT_FILE, scopes=drive_SCOPES)
    return creds

def upload_image_to_drive(image_path):
    """Upload image to Google Drive and make it publicly accessible"""
    file_metadata = {
        'name': image_path.split('/')[-1],
        'parents': [drive_PARENT_FOLDER_ID]
    }
    
    creds = authenticate()
    drive_service = build('drive', 'v3', credentials=creds)

    media = MediaFileUpload(image_path, mimetype='image/png')
    file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()
    
    file_id = file['id']
    print(f"File uploaded successfully. File ID: {file_id}")
    
    # Make the file publicly accessible
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }
    drive_service.permissions().create(
        fileId=file_id,
        body=permission
    ).execute()
    print(f"File {file_id} is now publicly accessible")
    
    return file_id

def create_new_slide(presentation_id):
    """Create a new blank slide and return its page ID"""
    creds = authenticate()
    slides_service = build('slides', 'v1', credentials=creds)
    
    # Get current presentation to find insertion index
    presentation = slides_service.presentations().get(presentationId=presentation_id).execute()
    slides = presentation.get('slides', [])
    
    # Create a new slide request
    slide_id = f"slide_{len(slides) + 1}"
    requests = [
        {
            'createSlide': {
                'objectId': slide_id,
                'insertionIndex': len(slides),  # Insert at the end
                'slideLayoutReference': {
                    'predefinedLayout': 'BLANK'
                }
            }
        }
    ]
    
    # Execute the requests
    body = {'requests': requests}
    response = slides_service.presentations().batchUpdate(
        presentationId=presentation_id, body=body).execute()
    
    print(f"Created new slide with ID: {slide_id}")
    return slide_id

def calculate_image_dimensions(image_percentage=0.90):
    """Calculate image dimensions as percentage of slide HEIGHT, maintaining aspect ratio"""
    # Calculate target height (90% of slide height)
    target_height = SLIDE_HEIGHT_PT * image_percentage
    
    return target_height

def get_image_aspect_ratio(image_path):
    """Get the aspect ratio of an image file"""
    try:
        from PIL import Image
        with Image.open(image_path) as img:
            width, height = img.size
            return width / height
    except ImportError:
        print("PIL not available, using default aspect ratio")
        return 16/9  # Default to 16:9
    except Exception as e:
        print(f"Could not get image dimensions: {e}")
        return 16/9  # Default to 16:9

def add_image_to_slide(file_id, presentation_id, page_id, image_path, image_percentage=0.90, title_text=""):
    """Add image from Drive to a specific slide, sized to percentage of slide height"""
    # Generate the public URL for the image
    image_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    
    # Get image aspect ratio
    image_aspect_ratio = get_image_aspect_ratio(image_path)
    
    # Calculate dimensions based on slide height percentage
    target_height = calculate_image_dimensions(image_percentage)
    final_height = target_height
    final_width = target_height * image_aspect_ratio
    
    # Calculate positions - leave space on left for text
    text_box_width = 120  # Width for text box on left
    margin = 20  # Margin between text and image
    
    # Position image on the right side
    image_x_pos = text_box_width + margin
    image_y_pos = (SLIDE_HEIGHT_PT - final_height) / 2
    
    print(f"Image dimensions: {final_width:.1f} x {final_height:.1f} PT")
    print(f"Image position: ({image_x_pos:.1f}, {image_y_pos:.1f})")
    print(f"Slide coverage: {(final_width/SLIDE_WIDTH_PT)*100:.1f}% width, {(final_height/SLIDE_HEIGHT_PT)*100:.1f}% height")
    
    # Authenticate and build the Slides service
    creds = authenticate()
    slides_service = build('slides', 'v1', credentials=creds)
    
    # Create requests list
    requests = []
    
    # Add text box if title is provided
    if title_text:
        text_box_id = f"textbox_{page_id}"
        # Create text box on the left side
        requests.append({
            'createShape': {
                'objectId': text_box_id,
                'shapeType': 'TEXT_BOX',
                'elementProperties': {
                    'pageObjectId': page_id,
                    'size': {
                        'width': {'magnitude': text_box_width, 'unit': 'PT'},
                        'height': {'magnitude': final_height, 'unit': 'PT'}
                    },
                    'transform': {
                        'scaleX': 1,
                        'scaleY': 1,
                        'translateX': 10,  # Small margin from left edge
                        'translateY': image_y_pos,  # Align with image
                        'unit': 'PT'
                    }
                }
            }
        })
        
        # Insert text into the text box
        requests.append({
            'insertText': {
                'objectId': text_box_id,
                'text': title_text
            }
        })
        
        # Format the text (Tahoma font, black color)
        requests.append({
            'updateTextStyle': {
                'objectId': text_box_id,
                'style': {
                    'fontFamily': 'Tahoma',
                    'fontSize': {
                        'magnitude': 14,
                        'unit': 'PT'
                    },
                    'foregroundColor': {
                        'opaqueColor': {
                            'rgbColor': {
                                'red': 0.0,
                                'green': 0.0,
                                'blue': 0.0
                            }
                        }
                    }
                },
                'fields': 'fontFamily,fontSize,foregroundColor'
            }
        })
        
        # Center align the text vertically and horizontally
        requests.append({
            'updateParagraphStyle': {
                'objectId': text_box_id,
                'style': {
                    'alignment': 'CENTER'
                },
                'fields': 'alignment'
            }
        })
    
    # Add the image
    requests.append({
        'createImage': {
            'url': image_url,
            'elementProperties': {
                'pageObjectId': page_id,
                'size': {
                    'width': {'magnitude': final_width, 'unit': 'PT'},
                    'height': {'magnitude': final_height, 'unit': 'PT'}
                },
                'transform': {
                    'scaleX': 1,
                    'scaleY': 1,
                    'translateX': image_x_pos,
                    'translateY': image_y_pos,
                    'unit': 'PT'
                }
            }
        }
    })
    
    # Execute the batch update request
    body = {'requests': requests}
    try:
        response = slides_service.presentations().batchUpdate(
            presentationId=presentation_id, body=body).execute()
        if title_text:
            print(f"✅ Image and text box added to slide {page_id}")
        else:
            print(f"✅ Image added to slide {page_id}")
        return response
    except Exception as e:
        print(f"❌ Error adding image to slide: {e}")
        return None

def get_or_create_target_slide(presentation_id, create_new=True):
    """Get existing slide 2 or create a new slide for the image"""
    creds = authenticate()
    slides_service = build('slides', 'v1', credentials=creds)
    
    # Get current slides
    presentation = slides_service.presentations().get(presentationId=presentation_id).execute()
    slides = presentation.get('slides', [])
    
    print(f"Presentation currently has {len(slides)} slide(s)")
    
    if create_new or len(slides) < 2:
        # Create a new slide
        page_id = create_new_slide(presentation_id)
        slide_number = len(slides) + 1
    else:
        # Use existing second slide
        page_id = slides[1]['objectId']  # Second slide (index 1)
        slide_number = 2
        print(f"Using existing slide 2 with ID: {page_id}")
    
    return page_id, slide_number

def upload_and_add_image(image_path, presentation_id, image_percentage=0.90, 
                        create_new_slide=True, title_text=""):
    """Complete workflow: upload image to Drive and add to appropriate slide"""
    try:
        # Step 1: Upload image to Drive
        file_id = upload_image_to_drive(image_path)
        
        # Step 2: Get or create target slide
        page_id, slide_number = get_or_create_target_slide(
            presentation_id, create_new_slide)
        
        # Step 3: Add image to slide with optional text box
        response = add_image_to_slide(file_id, presentation_id, page_id, 
                                    image_path, image_percentage, title_text)
        
        if response:
            print(f"✅ Image successfully added to slide {slide_number}!")
            return file_id, page_id
        else:
            print("❌ Failed to add image to presentation")
            return None, None
            
    except Exception as e:
        print(f"❌ Error in upload_and_add_image: {e}")
        return None, None



In [14]:
# Example usage
image_path = '/Volumes/Work/Coding/Github - Local/monthly_trades_tracking/images/NSE_NIFTY_1D_2025-09-08.png'

# Upload and add image - will create new slide if needed
file_id, page_id = upload_and_add_image(
    image_path=image_path,
    presentation_id=PRESENTATION_ID,
    image_percentage=0.90,         # 90% of slide height
    create_new_slide=True,         # Always create new slide
    title_text="NSE NIFTY\nAnalysis"  # Text for left side text box
)

if file_id:
    print(f"\n📊 Summary:")
    print(f"Drive file ID: {file_id}")
    print(f"Slide ID: {page_id}")
    print(f"Direct Drive link: https://drive.google.com/file/d/{file_id}/view")
    print(f"Presentation link: https://docs.google.com/presentation/d/{PRESENTATION_ID}/edit")

File uploaded successfully. File ID: 11ZRKMgPYsKMBHKBJaKUdBBRwoyYl_qaw
File 11ZRKMgPYsKMBHKBJaKUdBBRwoyYl_qaw is now publicly accessible
Presentation currently has 4 slide(s)
Created new slide with ID: slide_5
Image dimensions: 486.0 x 364.5 PT
Image position: (140.0, 20.2)
Slide coverage: 67.5% width, 90.0% height
✅ Image and text box added to slide slide_5
✅ Image successfully added to slide 5!

📊 Summary:
Drive file ID: 11ZRKMgPYsKMBHKBJaKUdBBRwoyYl_qaw
Slide ID: slide_5
Direct Drive link: https://drive.google.com/file/d/11ZRKMgPYsKMBHKBJaKUdBBRwoyYl_qaw/view
Presentation link: https://docs.google.com/presentation/d/1h6rz2eed078IJi4ouUhmaJem-4QooDZxe_izoWwqQoQ/edit
